# Autorzy
- Mateusz Łopaciński
- Mateusz Mazur

# Wczytanie danych

In [1]:
from scipy.io import loadmat

data = loadmat('./data/leukemia.mat')
X = data['X']
y = data['Y'].ravel()

# Usuwanie cech charakteryzujących się niską wariancją

## Import potrzebnych bibliotek

In [2]:
import numpy as np
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

## Usuwanie cech o niskiej wariancji

Dzielimy oryginalny zbór na zbiór treningowy i testowy

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

Wybieramy próg, dla którego obserwujemy zmianę skuteczności predykcji (metodą prób i błędów, obserwując zmiany wartości `accuracy`)

In [4]:
threshold = 3

Usuwamy z wyjściowego zbioru cechy o niskiej wariancji i dzielimy powstały zbiór na zbiór treningowy i testowy

In [5]:
selector = VarianceThreshold(threshold)
X_transformed = selector.fit_transform(X)

X_train_high_variance, X_test_high_variance, y_train, y_test = train_test_split(X_transformed, y, test_size=0.3, random_state=42)

## Trenowanie klasyfikatora `Random Forest` dla obu zbiorów

Trenujemy klasyfikator Random Forest i wyznaczamy `accuracy` dla oryginalnego zbioru oraz zbioru bez cech o niskiej wariancji

In [6]:
# Original dataset
clf_original = RandomForestClassifier(random_state=42)
clf_original.fit(X_train, y_train.ravel())
y_pred_original = clf_original.predict(X_test)
accuracy_original = accuracy_score(y_test, y_pred_original)

# Dataset after removing low variance features
clf_high_variance = RandomForestClassifier(random_state=42)
clf_high_variance.fit(X_train_high_variance, y_train.ravel())
y_pred_high_variance = clf_high_variance.predict(X_test_high_variance)
accuracy_high_variance = accuracy_score(y_test, y_pred_high_variance)

## Porównanie skuteczności predykcji

Porównujemy otrzymane wyniki. Jak widać, usunięcie cech o niskiej wariancji, powoduje spadek skuteczności predykcji

In [7]:
print("Skuteczność na oryginalnym zbiorze: {:.2f}".format(accuracy_original * 100))
print("Skuteczność po usunięciu cech o niskiej wariancji: {:.2f}".format(accuracy_high_variance * 100))

Skuteczność na oryginalnym zbiorze: 100.00
Skuteczność po usunięciu cech o niskiej wariancji: 95.45


# Wybór `m` najlepszych cech

## Import potrzebnych bibliotek

In [8]:
from sklearn.feature_selection import RFE, RFECV
from sklearn.linear_model import LogisticRegression

## Wybór liczby cech ze zbioru treningowego

Wybieramy liczbę `m` cech, które będziemy chcieli wyznaczyć, przy pomocy algorytmu `RFE`. Stosujemy się do sugestii, że docelowa liczba cech nie powinna przekraczać $1/3$ cech ze zbioru treningowego.

In [9]:
n_instances = X_train_high_variance.shape[0]
m = n_instances // 3

## Trening klasyfikatorów

Korzystamy z klasyfikatora regresji logistycznej do wyznaczenia `m` najlepszych cech

In [10]:
estimator_lr = LogisticRegression()
rfe_lr = RFE(estimator=estimator_lr, n_features_to_select=m)
X_train_selected_lr = rfe_lr.fit_transform(X_train_high_variance, y_train.ravel())
X_test_selected_lr = rfe_lr.transform(X_test_high_variance)

W tym samym celu, wykorzystujemy klasyfikator lasów losowych

In [11]:
estimator_rf = RandomForestClassifier(random_state=42)
rfe_rf = RFE(estimator=estimator_rf, n_features_to_select=m)
X_train_selected_rf = rfe_rf.fit_transform(X_train_high_variance, y_train.ravel())
X_test_selected_rf = rfe_rf.transform(X_test_high_variance)

# Wpływ metryk na dokładność klasyfikacji

## Import potrzebnych bibliotek

In [12]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold, cross_val_score

## Badanie wpływy metryk na dokładność klasyfikacji

Korzystamy z 4-krotnej walidacji krzyżowej oraz obu wytrenowanych wcześniej klasyfikatorów. Wyznaczamy dokładność, przy pomocy metryki `AUC` oraz metryki `accuracy`

In [13]:
n_splits = 4

cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

mean_accuracy_lr = np.mean(cross_val_score(rfe_lr, X_transformed, y.ravel(), cv=cv, scoring='accuracy'))
mean_accuracy_rf = np.mean(cross_val_score(rfe_rf, X_transformed, y.ravel(), cv=cv, scoring='accuracy'))

mean_auc_lr = np.mean(cross_val_score(rfe_lr, X_transformed, y.ravel(), cv=cv, scoring='roc_auc'))
mean_auc_rf = np.mean(cross_val_score(rfe_rf, X_transformed, y.ravel(), cv=cv, scoring='roc_auc'))

## Porównanie wyników

Jak widzimy, korzystając z metryki `AUC`, otrzymujemy dużo wyższe wyniki w przypadku obu klasyfikatorów niż dla metryki dokładności (`accuracy`). W przypadku lasów losowych oraz regresji logistycznej otrzymaliśmy takie same wyniki dla obu klasyfikatorów

In [14]:
print("Średnia dokładność dla regresji logistycznej: {:.2f}".format(mean_accuracy_lr * 100))
print("Średnia dokładność dla lasów losowych: {:.2f}".format(mean_accuracy_rf * 100))

print("Średnia metryka AUC dla regresji logistycznej: {:.2f}".format(mean_auc_lr * 100))
print("Średnia metryka AUC dla lasów losowych: {:.2f}".format(mean_auc_rf * 100))

Średnia dokładność dla regresji logistycznej: 93.06
Średnia dokładność dla lasów losowych: 93.06
Średnia metryka AUC dla regresji logistycznej: 98.29
Średnia metryka AUC dla lasów losowych: 99.31


# Porównanie dokładności wbudowanymi metodami selekcji cech

## Import potrzebnych bibliotek

In [15]:
from sklearn.feature_selection import SelectFromModel

## Selekcja cech wbudowanymi metodami

### Trening klasyfikatorów

#### Regresja logistyczna

In [16]:
estimator_lr = LogisticRegression(solver='saga', penalty='l1', C=0.1, max_iter=1000)
mean_accuracy_lr_l1 = np.mean(cross_val_score(estimator_lr, X_transformed, y.ravel(), cv=cv, scoring='accuracy'))

#### Lasy losowe

In [17]:
estimator_rf = RandomForestClassifier(random_state=42)
selector_rf = SelectFromModel(estimator_rf)
X_transformed_rf = selector_rf.fit_transform(X_transformed, y.ravel())
mean_accuracy_rf_se = np.mean(cross_val_score(estimator_rf, X_transformed_rf, y.ravel(), cv=cv, scoring='accuracy'))

## Porównanie wyników

In [18]:
print("Dokładność regresji logistycznej: {:.2f}".format(mean_accuracy_lr * 100))
print("Dokładność lasów losowych: {:.2f}".format(mean_accuracy_rf * 100))
print("Dokładność regresji logistycznej z regularyzacją L1: {:.2f}".format(mean_accuracy_lr_l1 * 100))
print("Dokładność lasów losowych z metodą ważności cech: {:.2f}".format(mean_accuracy_rf_se * 100))

Dokładność regresji logistycznej: 93.06
Dokładność lasów losowych: 93.06
Dokładność regresji logistycznej z regularyzacją L1: 88.89
Dokładność lasów losowych z metodą ważności cech: 93.06
